In [3]:

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model, model_from_json
import time
import pandas as pd

start = time.time()

#Define Path
sample_set = pd.read_csv("../Dataentry.csv")
model_weights_path = './models/weights.h5'
model_path = './models/CNN.best.from_scratch.hdf5'
test_path = 'data/alien_test'
diseases = ['Cardiomegaly','Emphysema','Effusion','Hernia','Nodule','Pneumothorax','Atelectasis','Pleural_Thickening','Mass','Edema','Consolidation','Infiltration','Fibrosis','Pneumonia','No Finding']
#diseases = diseases.sort()
diseases.sort()
labels = sample_set[['Image Index','Finding Labels']]

#Load the pre-trained models
#json_file = open('./models/model1.json','r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
loaded_model=load_model(model_path)
loaded_model.load_weights(model_weights_path)


#Prediction Function
def predict(file):
  x = load_img(file, target_size=(150,150))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = loaded_model.predict(x)
  #print (array)
  result = array[0]
  #print(result)
  answer = np.argmax(result)
  diss=[]
  vals=[]
  for i in range(len(result)):
    if result[i] > 0:
        diss.append(diseases[i])
        vals.append(result[i])
  
  final_op=sorted(zip(diss,vals), reverse=True)[:4]
  print ("Model Output: ",final_op)

  top5=[]
  for j in final_op:
        top5.append(j[0])
  return answer,top5

correct=0
wrong=0
total=0
#Walk the directory for every image
for i, ret in enumerate(os.walk(test_path)):
  for i, filename in enumerate(ret[2]):
    if filename.startswith("."):
      continue
    
    print(ret[0] + '/' + filename)
    total=total+1
    #get predicted result from model
    result,modellist = predict(ret[0] + '/' + filename)
    #print('Most Probable Predicted disease: ',diseases[result])
    #find actual value from dataset
    actual_output=labels[labels['Image Index']==filename]['Finding Labels']
    string_output=str(actual_output)
    spaced=string_output[string_output.find(" ")+1:string_output.find("\n")]
    actuallist=spaced.split("|")
    actuallist[0]=actuallist[0].lstrip()
    
    checking=0
    for i in modellist:
        if(i in actuallist):
            checking=1
            break
            
    if(checking==1):
        correct=correct+1
    elif(checking==0):
        wrong=wrong+1
    
    print('Actual output:',spaced)
    print ("-------------")
    
#print test results
print("\n\n\nNumber of correctly detected images: ",correct,"/",total)
print("Number of wrongly detected images: ",wrong,"/",total)
#Calculate execution time
end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
    print("Execution Time:",dur,"hours")


data/alien_test/00004798_008.png
Model Output:  [('Infiltration', 0.0019058826), ('Emphysema', 1.0931625e-26), ('Edema', 0.99809414), ('Consolidation', 7.055136e-34)]
Actual output:    Consolidation
-------------
data/alien_test/00017504_062.png
Model Output:  [('Infiltration', 2.8905251e-25), ('Emphysema', 5.0325478e-27), ('Edema', 1.0), ('Consolidation', 1.3925018e-12)]
Actual output:    Edema|Emphysema|Infiltration|Pneumonia
-------------
data/alien_test/00017618_019.png
Model Output:  [('Mass', 1.0), ('Hernia', 4.8246336e-29), ('Cardiomegaly', 3.6541671e-37)]
Actual output:    Cardiomegaly|Consolidation|Effusion|Mass|Pneum...
-------------
data/alien_test/00017641_008.png
Model Output:  [('Nodule', 1.4164025e-22), ('No Finding', 0.00052941847), ('Mass', 7.101149e-08), ('Infiltration', 1.0981116e-15)]
Actual output:    Consolidation|Effusion|Infiltration|Mass
-------------
data/alien_test/00017641_015.png
Model Output:  [('Nodule', 1.8136309e-34), ('No Finding', 1.3131693e-19), ('Ma

Actual output:    Mass|Pleural_Thickening
-------------
data/alien_test/00019899_025.png
Model Output:  [('Mass', 4.223087e-09), ('Infiltration', 0.9898289), ('Hernia', 0.010171039)]
Actual output:    Atelectasis|Consolidation|Effusion|Mass|Nodule...
-------------
data/alien_test/00019967_033.png
Model Output:  [('Edema', 1.7740955e-06), ('Consolidation', 0.9999982)]
Actual output:    Consolidation|Effusion
-------------
data/alien_test/00020213_093.png
Model Output:  [('No Finding', 4.577213e-29), ('Mass', 2.2945044e-34), ('Infiltration', 5.573074e-16), ('Fibrosis', 1.5778e-35)]
Actual output:    Atelectasis|Consolidation|Effusion|Infiltration
-------------
data/alien_test/00020213_110.png
Model Output:  [('No Finding', 2.8231805e-12), ('Mass', 1.7513695e-22), ('Infiltration', 1.3698906e-31), ('Emphysema', 3.1526678e-10)]
Actual output:    Cardiomegaly|Consolidation|Effusion
-------------
data/alien_test/00020326_021.png
Model Output:  [('Nodule', 1.5844703e-30), ('No Finding', 2.2660

Actual output:    Effusion|Infiltration
-------------
data/alien_test/00025556_013.png
Model Output:  [('Mass', 9.499173e-24), ('Hernia', 6.925205e-33), ('Effusion', 1.0), ('Atelectasis', 3.8540836e-36)]
Actual output:    Atelectasis|Effusion
-------------
data/alien_test/00025787_003.png
Model Output:  [('Infiltration', 0.00016795531), ('Emphysema', 1.4007803e-13), ('Edema', 0.99983203), ('Consolidation', 8.618604e-37)]
Actual output:    Effusion|Infiltration|Nodule
-------------
data/alien_test/00025787_012.png
Model Output:  [('No Finding', 4.0537904e-19), ('Mass', 8.565421e-27), ('Effusion', 1.0), ('Edema', 8.423138e-28)]
Actual output:    Effusion|Infiltration|Nodule
-------------
data/alien_test/00025958_001.png
Model Output:  [('Edema', 7.0342116e-17), ('Consolidation', 1.0), ('Cardiomegaly', 9.092872e-19)]
Actual output:    Consolidation
-------------
data/alien_test/00026132_010.png
Model Output:  [('No Finding', 7.91818e-35), ('Infiltration', 3.0932043e-07), ('Emphysema', 1.0